<a href="https://colab.research.google.com/github/Nili3005/ML/blob/main/Project_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
!pip install --upgrade ngboost
!pip install ucimlrepo

# Usage

We'll start with a probabilistic regression example on the Boston housing dataset:

In [68]:
import sys
sys.path.append('/Users/c242587/Desktop/projects/git/ngboost')

In [69]:
from ngboost import NGBRegressor

import pandas as pd
import numpy as np

#from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from ucimlrepo import fetch_ucirepo

#Fetch dataset
wine_quality = fetch_ucirepo(id=186)

X = wine_quality.data.features
Y = wine_quality.data.targets

#Flatten Y using ravel()
Y = np.ravel(Y)

#X, Y = load_boston(True)
#data_url = "http://lib.stat.cmu.edu/datasets/boston"
#raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
#X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
#Y = raw_df.values[1::2, 2]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

ngb = NGBRegressor().fit(X_train, Y_train)
Y_preds = ngb.predict(X_test)
Y_dists = ngb.pred_dist(X_test)

# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, Y_test)
print('Test MSE', test_MSE)

# test Negative Log Likelihood
test_NLL = -Y_dists.logpdf(Y_test).mean()
print('Test NLL', test_NLL)

[iter 0] loss=1.2878 val_loss=0.0000 scale=1.0000 norm=0.9177
[iter 100] loss=1.1094 val_loss=0.0000 scale=1.0000 norm=0.7897
[iter 200] loss=1.0362 val_loss=0.0000 scale=1.0000 norm=0.7637
[iter 300] loss=0.9956 val_loss=0.0000 scale=1.0000 norm=0.7548
[iter 400] loss=0.9698 val_loss=0.0000 scale=1.0000 norm=0.7502
Test MSE 0.4574650960932763
Test NLL 1.0008041686298093


Getting the estimated distributional parameters at a set of points is easy. This returns the predicted mean and standard deviation of the first five observations in the test set:

In [70]:
Y_dists[0:5].params

{'loc': array([5.61728423, 5.91117852, 5.50297282, 5.33244466, 5.95333415]),
 'scale': array([0.74652614, 0.58328679, 0.59505202, 0.50551899, 0.78658114])}

## Distributions

NGBoost can be used with a variety of distributions, broken down into those for regression (support on an infinite set) and those for classification (support on a finite set).

### Regression Distributions

| Distribution | Parameters | Implemented Scores | Reference |
| --- | --- | --- | --- |
| `Normal` | `loc`, `scale` | `LogScore`, `CRPScore` | [`scipy.stats` normal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) |
| `LogNormal` | `s`, `scale` | `LogScore`, `CRPScore` | [`scipy.stats` lognormal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lognorm.html) |
| `Exponential` | `scale` | `LogScore`, `CRPScore` | [`scipy.stats` exponential](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.expon.html) |

Regression distributions can be used through the `NGBRegressor()` constructor by passing the appropriate class as the `Dist` argument. `Normal` is the default.

In [71]:
from ngboost.distns import Exponential, Normal

#X, Y = load_boston(True)
X_reg_train, X_reg_test, Y_reg_train, Y_reg_test = train_test_split(X, Y, test_size=0.2)

ngb_norm = NGBRegressor(Dist=Normal, verbose=False).fit(X_reg_train, Y_reg_train)
ngb_exp = NGBRegressor(Dist=Exponential, verbose=False).fit(X_reg_train, Y_reg_train)

There are two prediction methods for `NGBRegressor` objects: `predict()`, which returns point predictions as one would expect from a standard regressor, and `pred_dist()`, which returns a distribution object representing the conditional distribution of $Y|X=x_i$ at the points $x_i$ in the test set.

In [72]:
ngb_norm.predict(X_reg_test)[0:5]

array([5.2344753 , 6.23909841, 5.6766842 , 5.66400779, 5.80232086])

In [73]:
ngb_exp.predict(X_reg_test)[0:5]

array([5.21600505, 6.28275926, 5.72739419, 5.6669637 , 5.78863671])

In [74]:
ngb_exp.pred_dist(X_reg_test)[0:5].params

{'scale': array([5.21600505, 6.28275926, 5.72739419, 5.6669637 , 5.78863671])}

#### Survival Regression

NGBoost supports analyses of right-censored data. Any distribution that can be used for regression in NGBoost can also be used for survival analysis in theory, but this requires the implementation of the right-censored version of the appropriate score. At the moment, `LogNormal` and `Exponential` have these scores implemented. To do survival analysis, use `NGBSurvival` and pass both the time-to-event (or censoring) and event indicator vectors to  `fit()`:

In [75]:
import numpy as np
from ngboost import NGBSurvival
from ngboost.distns import LogNormal

#X, Y = load_boston(True)
X_surv_train, X_surv_test, Y_surv_train, Y_surv_test = train_test_split(X, Y, test_size=0.2)

# introduce administrative censoring to simulate survival data
T_surv_train = np.minimum(Y_surv_train, 30) # time of an event or censoring
E_surv_train = Y_surv_train > 30 # 1 if T[i] is the time of an event, 0 if it's a time of censoring

ngb = NGBSurvival(Dist=LogNormal).fit(X_surv_train, T_surv_train, E_surv_train)

/usr/local/lib/python3.10/dist-packages/scipy/stats/_continuous_distns.py:6010: RuntimeWarning: overflow encountered in square
  lambda x, s: -np.log(x)**2 / (2*s**2) - np.log(s*x*np.sqrt(2*np.pi)),
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:108: RuntimeWarning: overflow encountered in exp
  self.scale = np.exp(params[1])
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:15: RuntimeWarning: invalid value encountered in multiply
  uncens = E * self.dist.logpdf(T)
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:109: RuntimeWarning: overflow encountered in exp
  self.dist = dist(s=self.scale, scale=np.exp(self.loc))
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:15: RuntimeWarning: invalid value encountered in multiply
  uncens = E * self.dist.logpdf(T)
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:109: RuntimeWarning: overflow encountered in exp
  self.dist = dist(s=self.scale, scale

[iter 0] loss=0.9862 val_loss=0.0000 scale=64.0000 norm=158.3494


/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:109: RuntimeWarning: overflow encountered in exp
  self.dist = dist(s=self.scale, scale=np.exp(self.loc))
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:15: RuntimeWarning: invalid value encountered in multiply
  uncens = E * self.dist.logpdf(T)
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:109: RuntimeWarning: overflow encountered in exp
  self.dist = dist(s=self.scale, scale=np.exp(self.loc))
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:15: RuntimeWarning: invalid value encountered in multiply
  uncens = E * self.dist.logpdf(T)
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:109: RuntimeWarning: overflow encountered in exp
  self.dist = dist(s=self.scale, scale=np.exp(self.loc))
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:15: RuntimeWarning: invalid value encountered in multiply
  uncens = E * self.dist.logpd

[iter 100] loss=0.0030 val_loss=0.0000 scale=8.0000 norm=0.3680


/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:109: RuntimeWarning: overflow encountered in exp
  self.dist = dist(s=self.scale, scale=np.exp(self.loc))
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:15: RuntimeWarning: invalid value encountered in multiply
  uncens = E * self.dist.logpdf(T)
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:109: RuntimeWarning: overflow encountered in exp
  self.dist = dist(s=self.scale, scale=np.exp(self.loc))
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:15: RuntimeWarning: invalid value encountered in multiply
  uncens = E * self.dist.logpdf(T)
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:109: RuntimeWarning: overflow encountered in exp
  self.dist = dist(s=self.scale, scale=np.exp(self.loc))
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:15: RuntimeWarning: invalid value encountered in multiply
  uncens = E * self.dist.logpd

[iter 200] loss=0.0008 val_loss=0.0000 scale=128.0000 norm=1.0183


/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:109: RuntimeWarning: overflow encountered in exp
  self.dist = dist(s=self.scale, scale=np.exp(self.loc))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_continuous_distns.py:6010: RuntimeWarning: divide by zero encountered in divide
  lambda x, s: -np.log(x)**2 / (2*s**2) - np.log(s*x*np.sqrt(2*np.pi)),
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:15: RuntimeWarning: invalid value encountered in multiply
  uncens = E * self.dist.logpdf(T)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_continuous_distns.py:6010: RuntimeWarning: divide by zero encountered in divide
  lambda x, s: -np.log(x)**2 / (2*s**2) - np.log(s*x*np.sqrt(2*np.pi)),
/usr/local/lib/python3.10/dist-packages/ngboost/distns/lognormal.py:15: RuntimeWarning: invalid value encountered in multiply
  uncens = E * self.dist.logpdf(T)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_continuous_distns.py:6010: RuntimeWarning

[iter 300] loss=-0.0000 val_loss=0.0000 scale=512.0000 norm=0.0938
== Quitting at iteration / GRAD 358


The scores currently implemented assume that the censoring is independent of survival, conditional on the observed predictors.

### Classification Distributions

| Distribution | Parameters | Implemented Scores | Reference |
| --- | --- | --- | --- |
| `k_categorical(K)` | `p0`, `p1`... `p{K-1}` | `LogScore` | [Categorical distribution on Wikipedia](https://en.wikipedia.org/wiki/Categorical_distribution) |
| `Bernoulli` | `p` | `LogScore` | [Bernoulli distribution on Wikipedia](https://en.wikipedia.org/wiki/Bernoulli_distribution) |

Classification distributions can be used through the `NGBClassifier()` constructor by passing the appropriate class as the `Dist` argument. `Bernoulli` is the default and is equivalent to `k_categorical(2)`.

In [76]:
from ngboost import NGBClassifier
from ngboost.distns import k_categorical, Bernoulli
from sklearn.datasets import load_breast_cancer

#X, y = load_breast_cancer(True)
data = load_breast_cancer()
X, y = data.data, data.target
y[0:15] = 2 # artificially make this a 3-class problem instead of a 2-class problem
X_cls_train, X_cls_test, Y_cls_train, Y_cls_test  = train_test_split(X, y, test_size=0.2)

ngb_cat = NGBClassifier(Dist=k_categorical(3), verbose=False) # tell ngboost that there are 3 possible outcomes
_ = ngb_cat.fit(X_cls_train, Y_cls_train) # Y should have only 3 values: {0,1,2}

When using NGBoost for classification, the outcome vector `Y` must consist only of integers from 0 to K-1, where K is the total number of classes. This is consistent with the classification standards in sklearn.

`NGBClassifier` objects have three prediction methods: `predict()` returns the most likely class, `predict_proba()` returns the class probabilities, and `pred_dist()` returns the distribution object.

In [77]:
ngb_cat.predict(X_cls_test)[0:5]

array([1, 0, 1, 1, 0])

In [78]:
ngb_cat.predict_proba(X_cls_test)[0:5]

array([[6.40659633e-03, 9.93236466e-01, 3.56937363e-04],
       [9.89288047e-01, 1.01615846e-02, 5.50368526e-04],
       [6.40659633e-03, 9.93236466e-01, 3.56937363e-04],
       [6.64372958e-03, 9.93005164e-01, 3.51106647e-04],
       [9.47529417e-01, 5.19434456e-02, 5.27137036e-04]])

In [79]:
ngb_cat.pred_dist(X_cls_test)[0:5].params

{'p0': array([0.0064066 , 0.98928805, 0.0064066 , 0.00664373, 0.94752942]),
 'p1': array([0.99323647, 0.01016158, 0.99323647, 0.99300516, 0.05194345]),
 'p2': array([0.00035694, 0.00055037, 0.00035694, 0.00035111, 0.00052714])}

## Scores

NGBoost supports the log score (`LogScore`, also known as negative log-likelihood) and CRPS (`CRPScore`), although each score may not be implemented for each distribution. The score is specified by the `Score` argument in the constructor.

In [80]:
from ngboost.scores import LogScore, CRPScore
from sklearn.metrics import accuracy_score

#NGBRegressor(Dist=Exponential, Score=CRPScore, verbose=False).fit(X_reg_train, Y_reg_train)
#NGBClassifier(Dist=k_categorical(3), Score=LogScore, verbose=False).fit(X_cls_train, Y_cls_train)

reg_model = NGBRegressor(Dist=Exponential, Score=CRPScore, verbose=False).fit(X_reg_train, Y_reg_train)
cls_model = NGBClassifier(Dist=k_categorical(3), Score=LogScore, verbose=False).fit(X_cls_train, Y_cls_train)

#For Regression model - Prediction and MSE for accuracy
reg_predictions= reg_model.predict(X_reg_test)
print(reg_predictions)

mse = mean_squared_error(Y_reg_test, reg_predictions)
print(f'Mean Squared Error: {mse}')


#For Classifier model - Prediction and accuracy
cls_predictions = cls_model.predict(X_cls_test)
print(cls_predictions)

predicted_labels = np.round(cls_predictions).astype(int)
accuracy = accuracy_score(Y_cls_test, predicted_labels)
print(f'Accuracy: {accuracy}')


[5.82349492 5.82665949 5.82349492 ... 5.82469643 5.82535707 5.82535707]
Mean Squared Error: 0.7159989168445375
[1 0 1 1 0 1 1 0 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0 0 1 1 1 1
 0 0 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 0 0 1 1 1 0 1 0 0 1 0 0 1
 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 0 1
 1 0 0]
Accuracy: 0.9210526315789473


## Base Learners

NGBoost can be used with any sklearn regressor as the base learner, specified with the `Base` argument. The default is a depth-3 regression tree.

In [81]:
from sklearn.tree import DecisionTreeRegressor

learner = DecisionTreeRegressor(criterion='friedman_mse', max_depth=5)

NGBSurvival(Dist=Exponential, Score=CRPScore, Base=learner, verbose=False).fit(X_surv_train, T_surv_train, E_surv_train)

NGBSurvival(Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=5),
            Dist=<class 'ngboost.distns.utils.SurvivalDistnClass.<locals>.SurvivalDistn'>,
            Score=<class 'ngboost.scores.CRPScore'>,
            random_state=RandomState(MT19937) at 0x79DDFA98F940, verbose=False)

## Other Arguments

The learning rate, number of estimators, minibatch fraction, and column subsampling are also easily adjusted:

In [82]:
ngb = NGBRegressor(n_estimators=100, learning_rate=0.01,
             minibatch_frac=0.5, col_sample=0.5)
ngb.fit(X_reg_train, Y_reg_train)

[iter 0] loss=1.2810 val_loss=0.0000 scale=1.0000 norm=0.9108


NGBRegressor(col_sample=0.5, minibatch_frac=0.5, n_estimators=100,
             random_state=RandomState(MT19937) at 0x79DDFA98F940)

Sample weights (for training) are set using the `sample_weight` argument to `fit`.

In [83]:
ngb = NGBRegressor(n_estimators=100, learning_rate=0.01,
             minibatch_frac=0.5, col_sample=0.5)
weights = np.random.random(Y_reg_train.shape)
ngb.fit(X_reg_train, Y_reg_train, sample_weight=weights)

[iter 0] loss=1.2772 val_loss=0.0000 scale=1.0000 norm=0.9210


NGBRegressor(col_sample=0.5, minibatch_frac=0.5, n_estimators=100,
             random_state=RandomState(MT19937) at 0x79DDFA98F940)